# A QIF analysis of Monty Hall

![](monty.png)


We play a game in which a prize is hidden behind one of three closed doors (the other two contain a goat).
- We choose one door,
- then the host opens a door __containing a goat__, among the two non-chosen ones.
- After seeing the goat we have the right to change our initial guess.

Should we?

How much information does this system leak?

A detailed description of the problem and its history can be found [here](https://en.wikipedia.org/wiki/Monty_Hall_problem).

In [1]:
import numpy as np
import qif

The secret in this system is the door in which the prize is hidden (0, 1 or 2).

Assuming the door containing the prize is chosen uniformly at random, before we start the game we have 1/3 chances of guessing the secret correctly.

In [11]:
pi = qif.probab.uniform(3)

print("Prior Bayes vulnerability", qif.measure.bayes_vuln.prior(pi))

Prior Bayes vulnerability 0.3333333333333333


We can model the game as the following channel `A`.
- __Input__: the door containing the prize (0,1 or 2)
- __Output__: the player's choice __and__ the door opened by the host.
  Eg. the output `0:1` means that the player chose door 0, and the host opened door 1.

The probabilities come directly from the rules of the game. Eg if the secret is `0`, the observation `1:0` can never happen because the host will never open the door containing the prize.

In [3]:
A = np.array([
	#0:1  0:2  1:0  1:2  2:0  2:1
	[1/6, 1/6,   0, 1/3,   0, 1/3],		# door 0
	[  0, 1/3, 1/6, 1/6, 1/3,   0],		# door 1
	[1/3,   0, 1/3,   0, 1/6, 1/6],		# door 2
])

This system clearly leaks information, since we learn that the door opened by the host is not the one we want, now there are only 2 possible doors instead of 3.

But is this __all__ we learn? Let compute the system's multiplicative Bayes leakage.

In [16]:
print("Mult Bayes leakage", qif.measure.bayes_vuln.mult_leakage(pi, A))


Mult Bayes leakage 2.0


A multiplicative leakage of 2 means that our probability of guessing the secret __doubles__ as a result of observing the system's output.
Before the observation it was 


The probability We can computet he probability of guessing correctly the secret after observing the output, which __turns out to be 2/3__!

In [ ]:
print("Posterior Bayes vulnerability", qif.measure.bayes_vuln.posterior(pi, A))

To see why, we compute below the best guess for each observable output (called a "strategy").

Let's consider the first output (`0:1`). Our best guess after seeing this output is the secret `2`.
In other words, if we choose `0` and the hosts open the door `1`, it is best to change our
guess and choose `2` instead.

We notice that this is true for all observations `X:Y`. The best guess is the door that is different than both `X` and `Y`.


In [6]:
print("Best guessing strategy", qif.measure.bayes_vuln.strategy(pi, A))

Best guessing strategy [2 1 2 0 1 0]


We can also easily compute the posterior probability of guessing correctly via the channel's __capacity__.

We know that:
- The posterior vulnerability $V_b(\pi, A)$ is equal to the prior vulnerability multiplied by the leakage $ \mathcal{L}^\times_b(\pi, A)$.
- For Bayes vulnerability, the capacity is given for the __uniform__ prior, so capacity and leakage coincide in our case.
- The capacity is easily computed as the sum of the column maxima of the channel.

So $V_b(\pi, A) = V_b(\pi) \mathcal{L}^\times_b(\pi, A) =V_b(\pi) \mathcal{ML}_b^\times(A)  = 2 \cdot\frac{1}{3} = \frac{2}{3}$

We verify this below.

In [14]:
capacity = qif.measure.bayes_vuln.mult_capacity(A)

print("Mult Bayes capacity", capacity)
print("Prior vuln. * capacity", qif.measure.bayes_vuln.prior(pi) * capacity)

Mult Bayes capacity 2.0
Prior vuln. * capacity 0.6666666666666666


### The

In [10]:
B = np.array([
	#?:0  ?:1  ?:2
	[  0, 1/2, 1/2],		# door 0
	[1/2,   0, 1/2],		# door 1
	[1/2, 1/2,   0],		# door 2
]);
print("Bayes vulnerability", qif.measure.bayes_vuln.posterior(pi, B))

print("Refinement?", qif.refinement.refined_by(A, B));

R = qif.channel.factorize(B, A)
print(R)
print(np.all(A.dot(R) == B))

Bayes vulnerability 0.5
Refinement? True
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
True
